In [6]:
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# Remember: library imports are ALWAYS at the top of the script, no exceptions!
import sqlite3
import os
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.impute import KNNImputer
from pandas_profiling import ProfileReport

In [7]:
data = pd.read_csv("donors.csv", header=0, sep=',')

/Users/sofia/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (9,412) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
data.head()

,Unnamed: 0,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,0,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,...,7.741935,95515,0,L,4,E,X,X,X,C
1,1,2014-01-01,BOA,1,CA,91326,,,1972-02-01,0,...,15.666667,148535,0,L,2,G,X,X,X,A
2,2,2010-01-01,AMH,1,NC,27017,,,NaN,0,...,7.481481,15078,1,L,4,E,X,X,X,C
3,3,2007-01-01,BRY,0,CA,95953,,,1948-01-01,0,...,6.812500,172556,1,L,4,E,X,X,X,C
4,4,2006-01-01,,0,FL,33176,,,1940-01-01,0,...,6.864865,7112,1,L,2,F,X,X,X,A


In [4]:
data.columns

Index(['Unnamed: 0', 'ODATEDW', 'OSOURCE', 'TCODE', 'STATE', 'ZIP', 'MAILCODE',
       'PVASTATE', 'DOB', 'NOEXCH',
       ...
       'AVGGIFT', 'CONTROLN', 'HPHONE_D', 'RFA_2R', 'RFA_2F', 'RFA_2A',
       'MDMAUD_R', 'MDMAUD_F', 'MDMAUD_A', 'GEOCODE2'],
      dtype='object', length=476)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Columns: 476 entries, Unnamed: 0 to GEOCODE2
dtypes: float64(49), int64(302), object(125)
memory usage: 346.5+ MB


# Data Preparation

## Problems:
- Duplicates?
- Data types?
- Missing values?
- Strange values?
- Descriptive statistics?


In [14]:
# replace "" by nans
data.replace("", np.nan, inplace=True)

In [15]:
data.dtypes

Unnamed: 0     int64
ODATEDW       object
OSOURCE       object
TCODE          int64
STATE         object
               ...  
RFA_2A        object
MDMAUD_R      object
MDMAUD_F      object
MDMAUD_A      object
GEOCODE2      object
Length: 476, dtype: object

## Missing values


### Data Reduction

In order to deal with missing values it was necessary to calculate it dimension (percentage) of each column in the data set. 
A copy of data was created to assess an overview of our data exploration in every step of the data cleansing. 
After the missing values assessment, the data set will have a reduction of features which percentage is above 30% in order to correct inconsistencies in data. 

In [17]:
data_central = data.copy()

In [25]:
# Percentage of missing values in each column
nan_percentage = data_central.isna().sum()/len(data_central)*100.00
nan_percentage

Unnamed: 0    0.000000
ODATEDW       0.000000
OSOURCE       0.000000
TCODE         0.000000
STATE         0.000000
                ...   
RFA_2A        0.000000
MDMAUD_R      0.000000
MDMAUD_F      0.000000
MDMAUD_A      0.000000
GEOCODE2      0.138347
Length: 476, dtype: float64

In [28]:
# Select the columns with a "nan" percentage above 30%
above_na = nan_percentage[nan_percentage>=30].index
above_na

Index(['NUMCHLD', 'WEALTH1', 'MBCRAFT', 'MBGARDEN', 'MBBOOKS', 'MBCOLECT',
       'MAGFAML', 'MAGFEM', 'MAGMALE', 'PUBGARDN', 'PUBCULIN', 'PUBHLTH',
       'PUBDOITY', 'PUBNEWFN', 'PUBPHOTO', 'PUBOPP', 'WEALTH2', 'ADATE_5',
       'ADATE_10', 'ADATE_13', 'ADATE_15', 'ADATE_20', 'ADATE_21', 'ADATE_23',
       'ADATE_24', 'RDATE_3', 'RDATE_4', 'RDATE_5', 'RDATE_6', 'RDATE_7',
       'RDATE_8', 'RDATE_9', 'RDATE_10', 'RDATE_11', 'RDATE_12', 'RDATE_13',
       'RDATE_14', 'RDATE_15', 'RDATE_16', 'RDATE_17', 'RDATE_18', 'RDATE_19',
       'RDATE_20', 'RDATE_21', 'RDATE_22', 'RDATE_23', 'RDATE_24', 'RAMNT_3',
       'RAMNT_4', 'RAMNT_5', 'RAMNT_6', 'RAMNT_7', 'RAMNT_8', 'RAMNT_9',
       'RAMNT_10', 'RAMNT_11', 'RAMNT_12', 'RAMNT_13', 'RAMNT_14', 'RAMNT_15',
       'RAMNT_16', 'RAMNT_17', 'RAMNT_18', 'RAMNT_19', 'RAMNT_20', 'RAMNT_21',
       'RAMNT_22', 'RAMNT_23', 'RAMNT_24'],
      dtype='object')

In [29]:
# Check the number of columns above 30%
len(above_na)

69

In [31]:
# Drop the list above_na which contains the columns names of data 
data_central = data_central.drop(columns=above_na)

KeyError: "['NUMCHLD' 'WEALTH1' 'MBCRAFT' 'MBGARDEN' 'MBBOOKS' 'MBCOLECT' 'MAGFAML'\n 'MAGFEM' 'MAGMALE' 'PUBGARDN' 'PUBCULIN' 'PUBHLTH' 'PUBDOITY' 'PUBNEWFN'\n 'PUBPHOTO' 'PUBOPP' 'WEALTH2' 'ADATE_5' 'ADATE_10' 'ADATE_13' 'ADATE_15'\n 'ADATE_20' 'ADATE_21' 'ADATE_23' 'ADATE_24' 'RDATE_3' 'RDATE_4' 'RDATE_5'\n 'RDATE_6' 'RDATE_7' 'RDATE_8' 'RDATE_9' 'RDATE_10' 'RDATE_11' 'RDATE_12'\n 'RDATE_13' 'RDATE_14' 'RDATE_15' 'RDATE_16' 'RDATE_17' 'RDATE_18'\n 'RDATE_19' 'RDATE_20' 'RDATE_21' 'RDATE_22' 'RDATE_23' 'RDATE_24'\n 'RAMNT_3' 'RAMNT_4' 'RAMNT_5' 'RAMNT_6' 'RAMNT_7' 'RAMNT_8' 'RAMNT_9'\n 'RAMNT_10' 'RAMNT_11' 'RAMNT_12' 'RAMNT_13' 'RAMNT_14' 'RAMNT_15'\n 'RAMNT_16' 'RAMNT_17' 'RAMNT_18' 'RAMNT_19' 'RAMNT_20' 'RAMNT_21'\n 'RAMNT_22' 'RAMNT_23' 'RAMNT_24'] not found in axis"

### Fill Missing Values

In order to fill the missing values, we have to divide the data into metric and non-metric features to 

In [38]:
# Check the data types in data
data.dtypes.unique()


array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

In [43]:
# Check the object columns = non-metric columns
non_metric_features = data_central.select_dtypes(include=['object']).columns

# Drop the non metric features
metric_features = data_central.columns.drop(non_metric_features).to_list()

In [34]:
# Calculate the sum of missing values
na_sum = data_central.isna().sum()

#Calculate the median of each quantitive feature
na_median = data_central.median()


In [49]:
# Não tenho a certeza disto
# Get the mode(s) of each element along the non-metric features.
# The mode of a set of values is the value that appears most often. It can be multiple values.
modes = data_central[non_metric_features]
modes

,ODATEDW,OSOURCE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,RECP3,...,MAXRDATE,LASTDATE,FISTDATE,NEXTDATE,RFA_2R,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,2009-01-01,GRI,IL,61081,,,1957-12-01,0,,,...,2014-02-01,2015-12-01,2009-11-01,2010-03-01,L,E,X,X,X,C
1,2014-01-01,BOA,CA,91326,,,1972-02-01,0,,,...,2015-12-01,2015-12-01,2013-10-01,2015-04-01,L,G,X,X,X,A
2,2010-01-01,AMH,NC,27017,,,NaN,0,,,...,2012-07-01,2015-12-01,2010-01-01,2011-01-01,L,E,X,X,X,C
3,2007-01-01,BRY,CA,95953,,,1948-01-01,0,,,...,2014-11-01,2015-12-01,2007-02-01,2007-11-01,L,E,X,X,X,C
4,2006-01-01,,FL,33176,,,1940-01-01,0,X,X,...,2016-01-01,2016-01-01,1999-03-01,2000-05-01,L,F,X,X,X,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,2016-01-01,ASE,AK,99504,,,NaN,0,,,...,2016-02-01,2016-02-01,2016-02-01,NaN,L,G,X,X,X,C
95408,2016-01-01,DCD,TX,77379,,,1970-01-01,0,,,...,2016-03-01,2016-03-01,2016-03-01,NaN,L,F,X,X,X,A
95409,2015-01-01,MBC,MI,48910,,,1958-01-01,0,,X,...,2015-01-01,2016-10-01,2014-10-01,2015-01-01,L,E,X,X,X,B
95410,2006-01-01,PRV,CA,91320,,,1960-05-01,0,X,,...,2016-08-01,2017-01-01,2006-12-01,2007-04-01,L,F,X,X,X,A


Filling the non-metric and metric missing values

In [51]:
data_central.fillna(data_central.median(), inplace=True)
data_central.fillna(modes, inplace=True)
#data_central.isna().sum()  # checking how many NaNs we still have

Unnamed: 0      0
ODATEDW         0
OSOURCE         0
TCODE           0
STATE           0
             ... 
RFA_2A          0
MDMAUD_R        0
MDMAUD_F        0
MDMAUD_A        0
GEOCODE2      132
Length: 407, dtype: int64

In [ ]:
data.select_dtypes(include=['object']).columns